<a href="https://colab.research.google.com/github/Lococo231/MO_Tasks/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install pandas seaborn scikit-learn

# Домашнее задание 3

1. Выбрать номинальное значение как целевую переменную
2. Провести разведочный анализ **включая** анализ зависимости между номинальными переменными (chi2)
3. Выбрать `best_features`
4. Сделать нормализацию данных (scaler, onehotencoding)
4. Обучить модель логистической регрессии
5. Вывести метрики качества, описать полученный результат (также вывести коэфициенты модели)

In [26]:

import numpy as np
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)


df = pd.read_csv('dataset.csv')

categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

print(categorical_columns)


['Gender', 'Occupation', 'BMI Category', 'Blood Pressure', 'Sleep Disorder']


In [27]:

print(f"Размер: {df.shape}")
print(f"\nТипы данных:\n{df.dtypes}")
print(f"\nПропущенные значения:\n{df.isnull().sum()}")

categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()

print(f"НОМИНАЛЬНЫЕ ПЕРЕМЕННЫЕ: {categorical_columns}")
print(f"ЧИСЛОВЫЕ ПЕРЕМЕННЫЕ: {numeric_columns}")

print("ОПИСАТЕЛЬНАЯ СТАТИСТИКА (числовые)")
print(df[numeric_columns].describe())

print("РАСПРЕДЕЛЕНИЕ НОМИНАЛЬНЫХ ПЕРЕМЕННЫХ")
for col in categorical_columns:
    print(f"\n{col}:")
    print(df[col].value_counts())

print("АНАЛИЗ ЗАВИСИМОСТИ МЕЖДУ НОМИНАЛЬНЫМИ ПЕРЕМЕННЫМИ (CHI-2)")
for i, col1 in enumerate(categorical_columns):
    for col2 in categorical_columns[i+1:]:
        contingency_table = pd.crosstab(df[col1], df[col2])
        chi2, p_value, dof, expected = chi2_contingency(contingency_table)
        cramers_v = np.sqrt(chi2 / (len(df) * (min(contingency_table.shape) - 1)))

        print(f"\n{col1} vs {col2}:")
        print(f"Chi-2 статистика: {chi2:.4f}")
        print(f"P-value: {p_value:.2e}")
        print(f"Cramér's V: {cramers_v:.4f}")

        if cramers_v > 0.5:
            strength = "СИЛЬНАЯ"
        elif cramers_v > 0.3:
            strength = "СРЕДНЯЯ"
        elif cramers_v > 0.1:
            strength = "СЛАБАЯ"
        else:
            strength = "ОЧЕНЬ СЛАБАЯ"

        print(f"Сила связи: {strength}")

#Chi-2 — насколько фактические данные отличаются от ожидаемых. Ищем, чтобы обнаружить связь.
#P-value — вероятность ошибки. P < 0.05 = связь значимая.
#Cramér's V — сила связи (0-1). Ищем, чтобы узнать, насколько сильно переменные связаны.

Размер: (374, 13)

Типы данных:
Person ID                    int64
Gender                      object
Age                          int64
Occupation                  object
Sleep Duration             float64
Quality of Sleep             int64
Physical Activity Level      int64
Stress Level                 int64
BMI Category                object
Blood Pressure              object
Heart Rate                   int64
Daily Steps                  int64
Sleep Disorder              object
dtype: object

Пропущенные значения:
Person ID                    0
Gender                       0
Age                          0
Occupation                   0
Sleep Duration               0
Quality of Sleep             0
Physical Activity Level      0
Stress Level                 0
BMI Category                 0
Blood Pressure               0
Heart Rate                   0
Daily Steps                  0
Sleep Disorder             219
dtype: int64
НОМИНАЛЬНЫЕ ПЕРЕМЕННЫЕ: ['Gender', 'Occupation', 'BMI Catego

### EDA

In [28]:

#Occupation vs Blood Pressure
#BMI Category vs Blood Pressure

best_features = ['Occupation', 'Blood Pressure', 'BMI Category']

df_best = df[best_features].copy()

# Парсим
df_best['Blood Pressure'] = df_best['Blood Pressure'].str.split('/').str[0].astype(int)

def categorize_bp(bp):
    if bp < 120:
        return 'Low'
    elif bp <= 140:
        return 'Normal'
    else:
        return 'High'

df_best['Blood Pressure'] = df_best['Blood Pressure'].apply(categorize_bp)

# Все колонки теперь категориальные
categorical_features = df_best.columns.tolist()

# OneHotEncoding
ohe = OneHotEncoder(sparse_output=False, drop=None)
X = pd.DataFrame(
    ohe.fit_transform(df_best),
    columns=ohe.get_feature_names_out(categorical_features)
)

print(X.columns.tolist())
print(X.head())


['Occupation_Accountant', 'Occupation_Doctor', 'Occupation_Engineer', 'Occupation_Lawyer', 'Occupation_Manager', 'Occupation_Nurse', 'Occupation_Sales Representative', 'Occupation_Salesperson', 'Occupation_Scientist', 'Occupation_Software Engineer', 'Occupation_Teacher', 'Blood Pressure_High', 'Blood Pressure_Low', 'Blood Pressure_Normal', 'BMI Category_Normal', 'BMI Category_Normal Weight', 'BMI Category_Obese', 'BMI Category_Overweight']
   Occupation_Accountant  Occupation_Doctor  Occupation_Engineer  \
0                    0.0                0.0                  0.0   
1                    0.0                1.0                  0.0   
2                    0.0                1.0                  0.0   
3                    0.0                0.0                  0.0   
4                    0.0                0.0                  0.0   

   Occupation_Lawyer  Occupation_Manager  Occupation_Nurse  \
0                0.0                 0.0               0.0   
1                0.0   

In [32]:
# Целевая переменная (нужна из исходных данных)
y = df['Sleep Disorder'].fillna('None').map({'Insomnia': 0, 'Sleep Apnea': 1, 'None': 2})

# Разделение с stratify
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Обучение логистической регрессии
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

# Предсказания
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Вероятности для ROC-AUC (если бинарная классификация)
y_train_proba = model.predict_proba(X_train)
y_test_proba = model.predict_proba(X_test)

print("ОБУЧАЮЩАЯ ВЫБОРКА")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
print(f"Precision: {precision_score(y_train, y_train_pred, average='weighted'):.4f}")
print(f"Recall: {recall_score(y_train, y_train_pred, average='weighted'):.4f}")
print(f"F1: {f1_score(y_train, y_train_pred, average='weighted'):.4f}")
if len(set(y_train)) == 2:
    print(f"ROC-AUC: {roc_auc_score(y_train, y_train_proba[:, 1]):.4f}")

print("ТЕСТОВАЯ ВЫБОРКА")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_test_pred, average='weighted'):.4f}")
print(f"Recall: {recall_score(y_test, y_test_pred, average='weighted'):.4f}")
print(f"F1: {f1_score(y_test, y_test_pred, average='weighted'):.4f}")
if len(set(y_test)) == 2:
    print(f"ROC-AUC: {roc_auc_score(y_test, y_test_proba[:, 1]):.4f}")

print("Matrix confusion (test):")
print(confusion_matrix(y_test, y_test_pred))

#Accuracy — доля правильных предсказаний из всех. Сколько % мы угадали.
#Precision — из всех, что мы предсказали как "положительный класс", сколько действительно положительные (не ошибок).
#Recall — из всех действительно положительных, сколько мы нашли (не пропустили).
#F1 — это гармоническое среднее Precision и Recall.

ОБУЧАЮЩАЯ ВЫБОРКА
Accuracy: 0.9042
Precision: 0.9042
Recall: 0.9042
F1: 0.9038
ТЕСТОВАЯ ВЫБОРКА
Accuracy: 0.8938
Precision: 0.8960
Recall: 0.8938
F1: 0.8939
Matrix confusion (test):
[[21  1  1]
 [ 2 19  3]
 [ 3  2 61]]


### Model

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
y = df['your_feature']
X = df[best_features]

KeyError: 'your_feature'

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
) # stratify - разделение данных с учетом распределнеия категориальной переменной

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)


In [ ]:
...